In [4]:
import glob
import random
import pandas as pd
import numpy as np
import os
import csv
import copy

In [5]:
def create_trials_total(version: str, cues: list):
    """Create trials for the Episodic extinction experiment, using 5 different stimuli for each trial:
        A context, two narrative stimuli (NSs), a conditioned stimulus (CS) and one unconditioned stimulus (US), with an associated sound.
        
        The experiment has 10 different conditions with 40 trials in total, thus each condition has 4 trials.
        In half of the trials, the US is negative, in the other half it is neutral.
        
        On day 2, two conditions are omitted as a control and the remaining 8 conditions are repeated. 
        In half of the repeated trials, the original context is presented, in the other half a new context is presented.
        
        On day 3, all the stimuli are CSs are presented again as a cue. Memory tests are performed using this cue.
        
        As an overview for the conditions:
        condition 1: continued conditioning, negative US
        condition 2: EXT, negative US
        condition 3: control, negative US 
        condition 4: EXT, neutral US
        condition 5: continued conditioning, neutral US
        condition 6: control, neutral US

        
        ToDo: Add "" to every value in the dictionary"""

    # Define the index possibilities of the stimuli
    CS_i = list(range(36))
    US_A_i = list(range(18))
    US_N_i = list(range(18)) 

    # Create lists with all stimulus 
    CS = sorted(glob.glob('stimulus_files/CS/*'))[:36]
    US_A = sorted(glob.glob('stimulus_files/US/negative*'))[:18]
    US_N = sorted(glob.glob('stimulus_files/US/neutral*'))[:18]
    US_sounds_A = sorted(glob.glob('stimulus_files/USsounds/negative*'))[:18]
    US_sounds_N = sorted(glob.glob('stimulus_files/USsounds/neutral*'))[:18]

    # Remove the directory from the values within the lists
    CS = [stim.split("/")[-1] for stim in CS]
    US_A = [stim.split("/")[-1] for stim in US_A]
    US_N = [stim.split("/")[-1] for stim in US_N]
    US_sounds_A = [stim.split("/")[-1] for stim in US_sounds_A]
    US_sounds_N = [stim.split("/")[-1] for stim in US_sounds_N]

    # Define the empty data dictionaries for each day
    stim_day1 = {
        'start': [], 
                        'CS': [], 
        'delim1': [], 
                        'US': [], 
        'delim2': [], 
                        'US_sound': [], 
        'delim3': [], 
                        'condition': [], 
        'delim4': [], 
                        'epsiode_nr': [], 
        'delim5': [], 
                        'valence': [], 
        'end': []
    }

    stim_day2 = copy.deepcopy(stim_day1)  # Create a copy of the day 1 structure for day 2

    stim_day3 = {
        'start': [], 
                        'CS': [], 
        'delim1': [], 
                        'US': [], 
        'delim2': [], 
                        'US_sound': [],
        'delim3': [],
                        'US_cue': [], 
        'delim4': [], 
                        'condition': [], 
        'delim5': [], 
                        'epsiode_nr': [], 
        'delim6': [], 
                        'valence': [],
        'end': []
    }

    # Additional dictionaries without 'start', 'delim', or 'end'
    stim_day1_clean = {key: [] for key in stim_day1 if key not in ['start', 'delim1', 'delim2', 'delim3', 'delim4', 'delim5', 'end']}
    stim_day2_clean = {key: [] for key in stim_day2 if key not in ['start', 'delim1', 'delim2', 'delim3', 'delim4', 'delim5', 'end']}
    stim_day3_clean = {key: [] for key in stim_day3 if key not in ['start', 'delim1', 'delim2', 'delim3', 'delim4', 'delim5', 'delim6', 'end']}

    episode_nr = 1  # Fixed episode number for all trials
    # Fill the trials for each day using two for loops, to make 10 conditions with 4 trials each
    for i in range(6):  # Repeat 10 times for 40 rows (10 * 4 = 40)

        for j in range(6):  # Each iteration adds 10 rows

            # Randomly select indexes for each stimulus set
            index_CS = CS_i.pop(random.randint(0, len(CS_i) - 1))
            
            for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                List['start'].append('{"')
                List['delim1'].append('", "')
                List['delim2'].append('", "')
                List['delim3'].append('", "')
                List['delim4'].append('", "')
                List['delim5'].append('", "')
                List['end'].append('"},')
                if 'delim6' in List:
                    List['delim6'].append('", "')

                List['condition'].append(i + 1)
                List['epsiode_nr'].append(episode_nr)
                CleanList['condition'].append(i + 1)
                CleanList['epsiode_nr'].append(episode_nr)
                
                List['CS'].append(CS[index_CS])
                CleanList['CS'].append(CS[index_CS])


            # Alternating between US_A and US_N for each 5 iterations, and add US sounds for day1 and day2
            if i < 3:
                index_USA = US_A_i.pop(random.randint(0, len(US_A_i) - 1))
                for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                    List['US'].append(US_A[index_USA])
                    List['valence'].append(1)
                    List['US_sound'].append(US_sounds_A[index_USA])
                    if List == stim_day3: 
                        List['US_cue'].append(cues[index_USA])
                    CleanList['US'].append(US_A[index_USA])
                    CleanList['valence'].append(1)
                    CleanList['US_sound'].append(US_sounds_A[index_USA])
                    if CleanList == stim_day3_clean: 
                        CleanList['US_cue'].append(cues[index_USA])
            else:
                index_USN = US_N_i.pop(random.randint(0, len(US_N_i) - 1))
                for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                    List['US'].append(US_N[index_USN])
                    List['valence'].append(2)
                    List['US_sound'].append(US_sounds_N[index_USN])
                    if List == stim_day3:
                        List['US_cue'].append(cues[index_USN+18])
                    CleanList['US'].append(US_N[index_USN])
                    CleanList['valence'].append(2)
                    CleanList['US_sound'].append(US_sounds_N[index_USN])
                    if CleanList == stim_day3_clean:
                        CleanList['US_cue'].append(cues[index_USN+18])
            
            episode_nr += 1

    # Create the DataFrames and take out the control conditions for day 2
    day1 = pd.DataFrame(stim_day1)
    stim_day2 = pd.DataFrame(stim_day2)
    day2_new = stim_day2[~stim_day2['condition'].isin([3, 6])]
    day2 = pd.DataFrame(day2_new)
    day3 = pd.DataFrame(stim_day3)

    day1_clean = pd.DataFrame(stim_day1_clean)
    stim_day2_clean = pd.DataFrame(stim_day2_clean)
    pd_day2_clean = stim_day2_clean[~stim_day2_clean['condition'].isin([3, 6])]
    day2_clean = pd.DataFrame(pd_day2_clean)
    day3_clean = pd.DataFrame(stim_day3_clean)

    # Save the DataFrames to text and TSV files
    if not os.path.exists('Stimsets/'):
        os.makedirs('Stimsets/')

    if os.path.exists(f'Stimsets/{version}_day1.txt') or os.path.exists(f'Stimsets/{version}_day2.txt') or os.path.exists(f'Stimsets/{version}_day3_Presentation.txt'):
        print(f'{version} already exists, delete it first to overwrite')
    else:
        # Save the combined columns to text files
        day1['combined'] = day1.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
        day2['combined'] = day2.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
        day3['combined'] = day3.apply(lambda row: ''.join(row.values.astype(str)), axis=1)

        with open(f'Stimsets/{version}_day1.txt', 'w') as f:
            f.write('\n'.join(day1['combined']) + '\n')

        with open(f'Stimsets/{version}_day2.txt', 'w') as f:
            f.write('\n'.join(day2['combined']) + '\n')

        with open(f'Stimsets/{version}_day3.txt', 'w') as f:
            f.write('\n'.join(day3['combined']) + '\n')

        # Save the clean DataFrames to TSV files
        day1_clean.to_csv(f'Stimsets/{version}_day1.tsv', sep='\t', index=False)
        day2_clean.to_csv(f'Stimsets/{version}_day2.tsv', sep='\t', index=False)
        day3_clean.to_csv(f'Stimsets/{version}_day3.tsv', sep='\t', index=False)


In [6]:
cues = ["Bathtub", "Ribcage", "No lips", "Gimace", "Riverbank", "Black eye",
        "Hood of car", "Weapon", "Fracture", "Toilet", "Eye", "Tongue", 
        "Debris", "Toe", "Tied up", "Saw", "Whip", "In the vet's hand",
        "Crawling", "Leash", "Flame", "Ginger hair", "Lake", "Books",
        "Road", "Forest", "Shoes", "Vegetables", "Slices", "Brush",
        "In the sky", "Cheese", "Bike", "Tool", "Pile of fruit", "Shiny fur"]    

# Create the trials for 10 different versions, Important: Only create new versions if necessary. It will overwrite the existing files. 

for i in range(10):
    version = "version" + str(i+1)
    create_trials_total(version, cues)

### This was previous version, just for Presentation experiment:

In [ ]:
def create_trials_old(version: str):
    """Create trials for the Episodic extinction experiment, using 5 different stimuli for each trial:
        A context, two narrative stimuli (NSs), a conditioned stimulus (CS) and one unconditioned stimulus (US), with an associated sound.
        
        The experiment has 10 different conditions with 40 trials in total, thus each condition has 4 trials.
        In half of the trials, the US is negative, in the other half it is neutral.
        
        On day 2, two conditions are omitted as a control and the remaining 8 conditions are repeated. 
        In half of the repeated trials, the original context is presented, in the other half a new context is presented.
        
        On day 3, all the stimuli are CSs are presented again as a cue. Memory tests are performed using this cue.
        
        As an overview for the conditions:
        condition 1: continued conditioning, negative US, same context
        condition 2: EXT, negative US, same context
        condition 3: continued conditioning, negative US, new context
        condition 4: EXT, negative US, new context
        condition 5: control, negative US
        condition 6: EXT, neutral US, same context
        condition 7: continued conditioning, neutral US, same context
        condition 8: EXT, neutral US, new context
        condition 9: continued conditioning, neutral US, new context
        condition 10: control, neutral US

        
        ToDo: Add "" to every value in the dictionary"""

    # Define the index possibilities of the stimuli
    contexts_i = list(range(56))
    NS_i = list(range(40))
    CS_i = list(range(40))
    US_A_i = list(range(20))
    US_N_i = list(range(20)) 

    # Create lists with all stimulus 
    contexts = sorted(glob.glob('stimulus_files/contexts_equalized/*A*'))[:56]
    NS = sorted(glob.glob('stimulus_files/NS_equalized/A*'))[:40]
    CS = sorted(glob.glob('stimulus_files/CS_equalized/*'))[:40]
    US_A = sorted(glob.glob('stimulus_files/US_equalized/negative*'))[:20]
    US_N = sorted(glob.glob('stimulus_files/US_equalized/neutral*'))[:20]
    US_sounds_A = sorted(glob.glob('stimulus_files/USsounds/negative*'))[:20]
    US_sounds_N = sorted(glob.glob('stimulus_files/USsounds/neutral*'))[:20]

    # Remove the directory from the values within the lists
    contexts = [stim.split("/")[-1] for stim in contexts]
    NS = [stim.split("/")[-1] for stim in NS]
    CS = [stim.split("/")[-1] for stim in CS]
    US_A = [stim.split("/")[-1] for stim in US_A]
    US_N = [stim.split("/")[-1] for stim in US_N]
    US_sounds_A = [stim.split("/")[-1] for stim in US_sounds_A]
    US_sounds_N = [stim.split("/")[-1] for stim in US_sounds_N]

    # Define the empty data dictionaries for each day
    stim_day1 = {
        'start': [], 'context': [], 'delim': [], 'NS': [], 'delim2': [], 'CS': [], 'delim3': [],
        'US': [], 'delim4': [], 'US_sound': [], 'delim5': [], 'condition': [], 'delim6': [],
        'trial': [], 'delim7': [], 'valence': [], 'delim8': [], 'epsiode_nr': [], 'end': []
    }

    stim_day2 = {
        'start': [], 'context': [], 'delim': [], 'CS': [], 'delim2': [], 'US': [], 'delim3': [],
        'US_sound': [], 'delim4': [], 'condition': [], 'delim5': [], 'trial': [], 'delim6': [],
        'valence': [], 'delim7': [], 'epsiode_nr': [], 'end': []
    }

    stim_day3 = {
        'start': [], 'NS': [], 'delim': [], 'CS': [], 'delim2': [], 'US': [], 'delim3': [],
        'US_sound': [], 'delim4': [], 'condition': [], 'delim5': [], 'trial': [], 'delim6': [],
        'valence': [], 'delim7': [], 'epsiode_nr': [], 'end': []
    }

    # Additional dictionaries without 'start', 'delim', or 'end'
    stim_day1_clean = {key: [] for key in stim_day1 if key not in ['start', 'delim', 'delim2', 'delim3', 'delim4', 'delim5', 'delim6', 'delim7', 'delim8', 'end']}
    stim_day2_clean = {key: [] for key in stim_day2 if key not in ['start', 'delim', 'delim2', 'delim3', 'delim4', 'delim5', 'delim6', 'delim7', 'end']}
    stim_day3_clean = {key: [] for key in stim_day3 if key not in ['start', 'delim', 'delim2', 'delim3', 'delim4', 'delim5', 'delim6', 'delim7', 'end']}

    episode_nr = 1  # Fixed episode number for all trials
    # Fill the trials for each day using two for loops, to make 10 conditions with 4 trials each
    for i in range(10):  # Repeat 10 times for 40 rows (10 * 4 = 40)

        for j in range(4):  # Each iteration adds 10 rows

            # Randomly select indexes for each stimulus set
            index_C = contexts_i.pop(random.randint(0, len(contexts_i) - 1))
            index_NS = NS_i.pop(random.randint(0, len(NS_i) - 1))
            index_CS = CS_i.pop(random.randint(0, len(CS_i) - 1))
            
            for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                List['start'].append('{"')
                List['delim'].append('", "')
                List['delim2'].append('", "')
                List['delim3'].append('", "')
                List['delim4'].append('", "')
                List['delim5'].append('", "')
                List['delim6'].append('", "')
                List['delim7'].append('", "')
                List['end'].append('"},')
                if 'delim8' in List:
                    List['delim8'].append('", "')

                List['condition'].append(i + 1)
                List['trial'].append(j + 1)
                List['epsiode_nr'].append(episode_nr)
                CleanList['condition'].append(i + 1)
                CleanList['trial'].append(j + 1)
                CleanList['epsiode_nr'].append(episode_nr)

                if List == stim_day1:
                    List['context'].append(contexts[index_C])
                    List['NS'].append(NS[index_NS])
                    List['CS'].append(CS[index_CS])
                    CleanList['context'].append(contexts[index_C])
                    CleanList['NS'].append(NS[index_NS])
                    CleanList['CS'].append(CS[index_CS])
                if List == stim_day2:
                    List['context'].append(contexts[index_C])
                    List['CS'].append(CS[index_CS])
                    CleanList['context'].append(contexts[index_C])
                    CleanList['CS'].append(CS[index_CS])
                if List == stim_day3:
                    List['NS'].append(NS[index_NS])
                    List['CS'].append(CS[index_CS])
                    CleanList['NS'].append(NS[index_NS])
                    CleanList['CS'].append(CS[index_CS])

            # Alternating between US_A and US_N for each 5 iterations, and add US sounds for day1 and day2
            if i < 5:
                index_USA = US_A_i.pop(random.randint(0, len(US_A_i) - 1))
                for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                    List['US'].append(US_A[index_USA])
                    List['valence'].append(1)
                    List['US_sound'].append(US_sounds_A[index_USA])
                    CleanList['US'].append(US_A[index_USA])
                    CleanList['valence'].append(1)
                    CleanList['US_sound'].append(US_sounds_A[index_USA])
            else:
                index_USN = US_N_i.pop(random.randint(0, len(US_N_i) - 1))
                for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                    List['US'].append(US_N[index_USN])
                    List['valence'].append(2)
                    List['US_sound'].append(US_sounds_N[index_USN])
                    CleanList['US'].append(US_N[index_USN])
                    CleanList['valence'].append(2)
                    CleanList['US_sound'].append(US_sounds_N[index_USN])

            if i % 10 + 1 in [3, 4, 8, 9]:  # If the condition is 3, 4, 8 or 9, change the context on day 2 
                index_C = contexts_i.pop(random.randint(0, len(contexts_i) - 1))
                stim_day2['context'][-1] = contexts[index_C]
                stim_day2_clean['context'][-1] = contexts[index_C]
            
            episode_nr += 1

    # Create the DataFrames and take out the control conditions for day 2
    day1 = pd.DataFrame(stim_day1)
    stim_day2 = pd.DataFrame(stim_day2)
    day2_new = stim_day2[~stim_day2['condition'].isin([5, 10])]
    day2 = pd.DataFrame(day2_new)
    day3 = pd.DataFrame(stim_day3)

    day1_clean = pd.DataFrame(stim_day1_clean)
    stim_day2_clean = pd.DataFrame(stim_day2_clean)
    pd_day2_clean = stim_day2_clean[~stim_day2_clean['condition'].isin([5, 10])]
    day2_clean = pd.DataFrame(pd_day2_clean)
    day3_clean = pd.DataFrame(stim_day3_clean)

    # Save the DataFrames to text and TSV files
    if not os.path.exists('Stimsets/'):
        os.makedirs('Stimsets/')

    if os.path.exists(f'Stimsets/{version}_day1.txt') or os.path.exists(f'Stimsets/{version}_day2.txt') or os.path.exists(f'Stimsets/{version}_day3_Presentation.txt'):
        print(f'{version} already exists, delete it first to overwrite')
    else:
        # Save the combined columns to text files
        day1['combined'] = day1.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
        day2['combined'] = day2.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
        day3['combined'] = day3.apply(lambda row: ''.join(row.values.astype(str)), axis=1)

        with open(f'Stimsets/{version}_day1.txt', 'w') as f:
            f.write('\n'.join(day1['combined']) + '\n')

        with open(f'Stimsets/{version}_day2.txt', 'w') as f:
            f.write('\n'.join(day2['combined']) + '\n')

        with open(f'Stimsets/{version}_day3.txt', 'w') as f:
            f.write('\n'.join(day3['combined']) + '\n')

        # Save the clean DataFrames to TSV files
        day1_clean.to_csv(f'Stimsets/{version}_day1.tsv', sep='\t', index=False)
        day2_clean.to_csv(f'Stimsets/{version}_day2.tsv', sep='\t', index=False)
        day3_clean.to_csv(f'Stimsets/{version}_day3.tsv', sep='\t', index=False)